In [1]:
import sys, getopt, os
import bonesis
import pandas as pd
from colomoto_jupyter import tabulate
import mpbn
import itertools
import math
import numpy
import networkx as nx
import pickle
#import ginsim
import re
import subprocess
import numpy as np
from matplotlib import pyplot as plt
from math import isnan
import clingo as asp
import mpbn

In [2]:
from copy import deepcopy 

In [3]:
sys.path.append('../py_src/')

import constraints
import funForBonesis

In [4]:
geneOrder = ["Egr1","Junb","Tal1","Bclaf1","Myc","Fli1","Gata2","Ikzf1","Spi1","Cebpa","Gata1","Klf1","Zfpm1","CDK46CycD","CIPKIP"]


#dataTable = pd.read_csv("../input/obsDataLarge.csv",index_col = 0)
dataTable = pd.read_csv("obsDataPhenoMut.csv",index_col = 0)
#dataTable.index = ['iHSC', 'srHSC', 'qHSC', 'pLymph',"pNeuMast","pMk","pEr","diff","diff2","zero"] 
dataTable
data = dataTable.to_dict("index")
clean_data = dict()
for o in data.keys():
    clean_data[o] = {k: data[o][k] for k in data[o] if not isnan(data[o][k])}

data= clean_data
#data["G0MkHSC"] = {"Klf1":0,'Junb': 1,'Tal1':1,'CIPKIP': 1,"Ikzf1":0,"Cebpa":0,"Spi1":0,"CDK46CycD":0}
#data["prolifNeuMast"] = {'Egr1': 0,'Junb': 0,'Tal1': 0,'Fli1': 0,'Spi1': 1,'Cebpa': 1,'Klf1': 0,'Zfpm1': 0.0,'CIPKIP': 0,"CDK46CycD":1}

obsData = pd.DataFrame.from_dict(data, orient="index").fillna('*')[geneOrder]

#Add some mutant obs

obsData = obsData[geneOrder].reindex(["iHSC", "srHSC", "qHSC","diff","pLymph","pNeuMast","pMk","pEr","zero","G0MkHSC",'prolifNeuMast'])


obsData = obsData.transpose()

obsDataView = obsData.style.apply(lambda x: ["background: black" if v == "*" else  "background: yellow" if v == 1 else "background: purple" for v in x], axis = 1)
obsDataView.set_precision(0)

,iHSC,srHSC,qHSC,diff,pLymph,pNeuMast,pMk,pEr,zero,G0MkHSC,prolifNeuMast
Egr1,0,*,1,0,0,0,0,0,0,*,0
Junb,0,0,1,0,0,0,0,0,0,1,0
Tal1,1,1,1,*,*,0,1,1,0,1,0
Bclaf1,*,*,0,1,*,0,*,*,0,*,*
Myc,0,0,*,1,*,*,*,*,0,*,*
Fli1,1,1,1,0,0,0,1,0,0,*,0
Gata2,*,0,1,*,0,*,*,*,0,*,*
Ikzf1,0,0,0,0,1,0,0,0,0,0,0
Spi1,0,0,0,1,1,1,0,0,0,0,1
Cebpa,0,0,0,0,0,1,0,0,0,0,1


In [5]:
with open("possible_final_solutions.p", "rb") as input_file:
    solutions = pickle.load(input_file)

In [6]:
table = pd.DataFrame(solutions)
table.sort_values(by=geneOrder, inplace=True)
table[geneOrder]

,Egr1,Junb,Tal1,Bclaf1,Myc,Fli1,Gata2,Ikzf1,Spi1,Cebpa,Gata1,Klf1,Zfpm1,CDK46CycD,CIPKIP
176,Gata2,Myc,Gata1&!Spi1,Myc,Cebpa&Bclaf1&Spi1,Junb|(Gata1&Fli1&!Klf1),(Egr1&!Zfpm1&!Gata1)|(Gata2&Junb&!Spi1),(Gata2&!Gata1)|(Ikzf1&!Gata1),(Cebpa&!Gata2)|(!Gata2&Spi1&!Gata1),(Gata2&!Zfpm1)|(!Ikzf1&!Zfpm1&Spi1),!Spi1|(Fli1&!Ikzf1),Gata1&!Fli1,Gata1,Myc&Bclaf1,Junb
149,Gata2,Myc,Gata1&!Spi1,Myc,Cebpa&Bclaf1&Spi1,Junb|(Gata1&Fli1&!Klf1),(Egr1&!Zfpm1&!Gata1)|(Gata2&Junb&!Spi1),(Gata2&!Gata1)|(Ikzf1&!Gata1),(Cebpa&!Gata2)|(!Gata2&Spi1&!Gata1),(Gata2&!Zfpm1)|(!Ikzf1&!Zfpm1&Spi1),!Spi1|(Fli1&!Ikzf1),Gata1&!Fli1,Gata1,Bclaf1|Myc,Junb
75,Gata2,Myc,Gata1&!Spi1,Myc,Cebpa&Bclaf1&Spi1,Junb|(Gata1&Fli1&!Klf1),(Egr1&!Zfpm1&!Gata1)|(Gata2&Junb&!Spi1),(Gata2&!Gata1)|(Ikzf1&!Gata1),(Cebpa&!Gata2)|(!Gata2&Spi1&!Gata1),(Gata2&!Zfpm1&!Ikzf1)|(!Ikzf1&!Zfpm1&Spi1),!Spi1|(Fli1&!Ikzf1),Gata1&!Fli1,Gata1,Myc&Bclaf1,Junb
174,Gata2,Myc,Gata1&!Spi1,Myc,Cebpa&Bclaf1&Spi1,Junb|(Gata1&Fli1&!Klf1),(Egr1&!Zfpm1&!Gata1)|(Gata2&Junb&!Spi1),(Gata2&!Gata1)|(Ikzf1&!Gata1),(Cebpa&!Gata2)|(!Gata2&Spi1&!Gata1),(Gata2&!Zfpm1&!Ikzf1)|(!Ikzf1&!Zfpm1&Spi1),!Spi1|(Fli1&!Ikzf1),Gata1&!Fli1,Gata1,Bclaf1|Myc,Junb
200,Gata2,Myc,Gata1&!Spi1,Myc,Cebpa&Bclaf1&Spi1,Junb|(Gata1&Fli1&!Klf1),(Egr1&!Zfpm1&!Gata1)|(Gata2&Junb&!Spi1),(Gata2&!Gata1)|(Ikzf1&!Gata1),(!Gata2&Cebpa&!Gata1)|(!Gata2&Spi1&!Gata1),(Gata2&!Zfpm1)|(!Ikzf1&Spi1),!Spi1|(Fli1&!Ikzf1),Gata1&!Fli1,Gata1,Myc&Bclaf1,Junb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Gata2,Myc,Gata1|!Spi1,Myc,Cebpa&Bclaf1&Spi1,(Junb&!Klf1)|(Gata1&Fli1&!Klf1),(Gata2&Junb&!Spi1)|(!Spi1&Egr1&!Zfpm1&!Gata1),(Gata2&!Gata1)|(Ikzf1&!Gata1),(Spi1&!Gata1)|(!Gata2&Cebpa&!Gata1),(Gata2&!Zfpm1)|(!Ikzf1&!Zfpm1&Spi1),!Spi1|(Fli1&!Ikzf1),Gata1&!Fli1,Gata1,Bclaf1|Myc,Junb
58,Gata2,Myc,Gata1|!Spi1,Myc,Cebpa&Bclaf1&Spi1,(Junb&!Klf1)|(Gata1&Fli1&!Klf1),(Gata2&Junb&!Spi1)|(!Spi1&Egr1&!Zfpm1&!Gata1),(Gata2&!Gata1)|(Ikzf1&!Gata1),(Spi1&!Gata1)|(!Gata2&Cebpa&!Gata1),(Gata2&!Zfpm1&!Ikzf1)|(!Ikzf1&!Zfpm1&Spi1),!Spi1|(Fli1&!Ikzf1),Gata1&!Fli1,Gata1,Myc&Bclaf1,Junb
92,Gata2,Myc,Gata1|!Spi1,Myc,Cebpa&Bclaf1&Spi1,(Junb&!Klf1)|(Gata1&Fli1&!Klf1),(Gata2&Junb&!Spi1)|(!Spi1&Egr1&!Zfpm1&!Gata1),(Gata2&!Gata1)|(Ikzf1&!Gata1),(Spi1&!Gata1)|(!Gata2&Cebpa&!Gata1),(Gata2&!Zfpm1&!Ikzf1)|(!Ikzf1&!Zfpm1&Spi1),!Spi1|(Fli1&!Ikzf1),Gata1&!Fli1,Gata1,Bclaf1|Myc,Junb
38,Gata2,Myc,Gata1|!Spi1,Myc,Cebpa&Bclaf1&Spi1,(Junb&!Klf1)|(Gata1&Fli1&!Klf1),(Gata2&Junb&!Spi1)|(!Spi1&Egr1&!Zfpm1&!Gata1),(Gata2&!Gata1)|(Ikzf1&!Gata1),(Spi1&!Gata1)|(!Gata2&Cebpa&!Gata1),(!Ikzf1&Spi1)|(Gata2&!Zfpm1&!Ikzf1),!Spi1|(Fli1&!Ikzf1),Gata1&!Fli1,Gata1,Myc&Bclaf1,Junb


In [7]:
table.nunique()

Bclaf1       1
CDK46CycD    2
CIPKIP       1
Cebpa        4
Egr1         1
Fli1         3
Gata1        1
Gata2        3
Ikzf1        1
Junb         1
Klf1         1
Myc          1
Spi1         6
Tal1         2
Zfpm1        1
dtype: int64

In [8]:
table.nunique().index

Index(['Bclaf1', 'CDK46CycD', 'CIPKIP', 'Cebpa', 'Egr1', 'Fli1', 'Gata1',
       'Gata2', 'Ikzf1', 'Junb', 'Klf1', 'Myc', 'Spi1', 'Tal1', 'Zfpm1'],
      dtype='object')

In [9]:
for n in table.nunique().index:
    if table.nunique()[n] > 1:
        print(n)
        print(pd.DataFrame(table[n].unique())) 


CDK46CycD
            0
0  Myc&Bclaf1
1  Bclaf1|Myc
Cebpa
                                            0
0         (Gata2&!Zfpm1)|(!Ikzf1&!Zfpm1&Spi1)
1  (Gata2&!Zfpm1&!Ikzf1)|(!Ikzf1&!Zfpm1&Spi1)
2                (Gata2&!Zfpm1)|(!Ikzf1&Spi1)
3         (!Ikzf1&Spi1)|(Gata2&!Zfpm1&!Ikzf1)
Fli1
                                 0
0          Junb|(Gata1&Fli1&!Klf1)
1        (Gata1&Fli1)|(Junb&!Klf1)
2  (Junb&!Klf1)|(Gata1&Fli1&!Klf1)
Gata2
                                               0
0        (Egr1&!Zfpm1&!Gata1)|(Gata2&Junb&!Spi1)
1        (Gata2&Junb)|(!Spi1&Egr1&!Zfpm1&!Gata1)
2  (Gata2&Junb&!Spi1)|(!Spi1&Egr1&!Zfpm1&!Gata1)
Spi1
                                            0
0         (Cebpa&!Gata2)|(!Gata2&Spi1&!Gata1)
1  (!Gata2&Cebpa&!Gata1)|(!Gata2&Spi1&!Gata1)
2                  Cebpa|(!Gata2&Spi1&!Gata1)
3         (Cebpa&!Gata1)|(!Gata2&Spi1&!Gata1)
4                (Cebpa&!Gata2)|(Spi1&!Gata1)
5         (Spi1&!Gata1)|(!Gata2&Cebpa&!Gata1)
Tal1
             0
0  Gata1&!Spi1
1  

In [11]:
keptRules = mpbn.MPBooleanNetwork({
    "Tal1": "Gata1&!Spi1",
    "Spi1": "(!Gata1&!Gata2&Cebpa)|(Spi1&!Gata2&!Gata1)",
    "Gata2": "(Junb&Gata2)|(!Zfpm1&Egr1&!Spi1&!Gata1)",
    "Fli1": "(!Klf1&Junb)|(!Klf1&Gata1&Fli1)",
    "Cebpa": "(Gata2&!Zfpm1&!Ikzf1)|(!Ikzf1&!Zfpm1&Spi1)",
    "CDK46CycD": "Myc&Bclaf1"
})
keptRules

CDK46CycD <- Myc&Bclaf1
Cebpa <- (Gata2&!Zfpm1&!Ikzf1)|(!Ikzf1&!Zfpm1&Spi1)
Fli1 <- (!Klf1&Junb)|(!Klf1&Gata1&Fli1)
Gata2 <- (Junb&Gata2)|(!Zfpm1&Egr1&!Spi1&!Gata1)
Spi1 <- (!Gata1&!Gata2&Cebpa)|(Spi1&!Gata2&!Gata1)
Tal1 <- Gata1&!Spi1

In [12]:
tableFinal = table.copy()
for n in table.nunique().index:
    if table.nunique()[n] > 1:
        print(n)
        tableFinal = tableFinal[(tableFinal[n] == keptRules[n])]

CDK46CycD
Cebpa
Fli1
Gata2
Spi1
Tal1


In [13]:
solutionFinal = solutions[tableFinal.index[0]] #### result order should be stable with the sorting
print(solutionFinal)

Bclaf1 <- Myc
CDK46CycD <- Myc&Bclaf1
CIPKIP <- Junb
Cebpa <- (Gata2&!Zfpm1&!Ikzf1)|(!Ikzf1&!Zfpm1&Spi1)
Egr1 <- Gata2
Fli1 <- (Junb&!Klf1)|(Gata1&Fli1&!Klf1)
Gata1 <- !Spi1|(Fli1&!Ikzf1)
Gata2 <- (Gata2&Junb)|(!Spi1&Egr1&!Zfpm1&!Gata1)
Ikzf1 <- (Gata2&!Gata1)|(Ikzf1&!Gata1)
Junb <- Myc
Klf1 <- Gata1&!Fli1
Myc <- Cebpa&Bclaf1&Spi1
Spi1 <- (!Gata2&Cebpa&!Gata1)|(!Gata2&Spi1&!Gata1)
Tal1 <- Gata1&!Spi1
Zfpm1 <- Gata1



In [14]:
sol = solutionFinal.to_biolqm()
sol = biolqm.to_ginsim(sol)
sol.save('solutionFinal0321.zginml')

In [17]:
def testPerturb(model,perturbations,data):
    test_mut = model.copy()
    for perturbation in perturbations:
        del test_mut[[*perturbation.keys()][0]]
        test_mut[[*perturbation.keys()][0]] = [*perturbation.values()][0]
        data_mut = deepcopy(data)
        if ([*perturbation.values()][0] == 1) | ([*perturbation.values()][0] == 0):
            data_mut["iHSC"][[*perturbation.keys()][0]] = [*perturbation.values()][0]
    print("reachability of quiescence state:")
    print(test_mut.reachability(data["iHSC"],data_mut["qHSC"]))
    print("reachability of SR state:")
    print(test_mut.reachability(data["iHSC"],data_mut["srHSC"]))
    att = pd.DataFrame(test_mut.attractors(reachable_from=data_mut["iHSC"]))[geneOrder]
    return att

In [20]:
testPerturb(solutionFinal,[{"Ikzf1":1},{"Junb":1},{"Cebpa":"(Spi1&!Zfpm1&!Ikzf1)"}],data)

reachability of quiescence state:
True
reachability of SR state:
False


,Egr1,Junb,Tal1,Bclaf1,Myc,Fli1,Gata2,Ikzf1,Spi1,Cebpa,Gata1,Klf1,Zfpm1,CDK46CycD,CIPKIP
0,0,1,1,0,0,1,0,1,0,0,1,0,1,0,1
1,0,1,1,0,0,0,0,1,0,0,1,1,1,0,1
2,1,1,1,0,0,1,1,1,0,0,1,0,1,0,1
3,1,1,1,0,0,0,1,1,0,0,1,1,1,0,1


In [ ]:
test_mut = solutionFinal.copy()
#test_mut["Junb"] = 1
#test_mut["Cebpa"] = "(Spi1&!Zfpm1&!Ikzf1)"
test_mut["Ikzf1"] = 1

print("reachability of quiescence state:")
print(test_mut.reachability(data["iHSC"],data["qHSC"]))
print("reachability of SR state:")
print(test_mut.reachability(data["iHSC"],data["srHSC"]))
attfromHSCNoEpo = pd.DataFrame(test_mut.attractors(reachable_from=data["iHSC"]))[geneOrder]
attfromHSCNoEpo

In [ ]:
testPerturb(solutionFinal,[{"Ikzf1":1},{"Junb":1},{"Cebpa":"(Spi1&!Zfpm1&!Ikzf1)"}],data)